# Code to fetch Historical stock data for past 1 year with the interval of 1 day:

In [ ]:
import yfinance as yf
import pandas as pd
import boto3
from datetime import datetime

# AWS S3 Configuration
#S3_BUCKET = "data608-2025-stock-market-data"
#S3_KEY = "historical-stock-data/stock_1y_1d.csv"
S3_BUCKET = "stock-market-data-uofc"
S3_KEY = "historical-data/stock_5y_1d.csv"
s3_client = boto3.client("s3")

# List of stocks to fetch
stocks = ["AAPL", "MSFT", "AMZN", "GOOGL", "META", "NVDA", "TSLA"]

# Fetch stock data from a given start date
def fetch_stock_data(start_date):
    all_stock_data = []
    for stock in stocks:
        print(f"Fetching {stock} from {start_date}...")
        ticker = yf.Ticker(stock)
        history = ticker.history(start=start_date, interval="1d")
        history.reset_index(inplace=True)
        history["Stock"] = stock
        history = history[["Stock", "Date", "Open", "High", "Low", "Close", "Volume"]]
        all_stock_data.append(history)
    return pd.concat(all_stock_data, ignore_index=True)

# Load data from S3
def load_existing_data():
    try:
        response = s3_client.get_object(Bucket=S3_BUCKET, Key=S3_KEY)
        existing_data = pd.read_csv(response['Body'], parse_dates=["Date"])
        print(f"Loaded existing data: {len(existing_data)} rows")
        return existing_data
    except s3_client.exceptions.NoSuchKey:
        print("No existing data found. Fetching fresh...")
        return None

# Save DataFrame to S3
def upload_to_s3(df):
    csv_buffer = df.to_csv(index=False)
    s3_client.put_object(Bucket=S3_BUCKET, Key=S3_KEY, Body=csv_buffer)
    print(f"Uploaded to S3: {S3_KEY}")

# Main logic
existing_df = load_existing_data()

if existing_df is not None:
    last_date = existing_df["Date"].max().date()
    next_date = last_date + pd.Timedelta(days=1)
    new_data = fetch_stock_data(start_date=next_date)
    combined_df = pd.concat([existing_df, new_data], ignore_index=True)
    combined_df.drop_duplicates(subset=["Stock", "Date"], inplace=True)
else:
    start_date = (datetime.now() - pd.Timedelta(days=365)).date()
    combined_df = fetch_stock_data(start_date=start_date)

upload_to_s3(combined_df)


# Code to fetch  Historical stock data of past 60 days with the interval of 5 minutes:

In [ ]:
import yfinance as yf
import pandas as pd
import boto3
from datetime import datetime, timedelta
from datetime import timezone
start_time = datetime.now(timezone.utc) - timedelta(days=60)

# AWS S3 Configuration
#S3_BUCKET = "data608-2025-stock-market-data"
S3_BUCKET = "stock-market-data-uofc"
S3_KEY = "real-time-stock-data/stock_60d_5m.csv"
s3_client = boto3.client("s3")

# Stocks to fetch
stocks = ["AAPL", "MSFT", "AMZN", "GOOGL", "META", "NVDA", "TSLA"]

# Function to fetch 5-minute interval data for the last 60 days
def fetch_stock_data(start_datetime):
    all_data = []
    for stock in stocks:
        print(f"Fetching {stock} from {start_datetime}...")
        ticker = yf.Ticker(stock)
        history = ticker.history(period="60d", interval="5m")
        history.reset_index(inplace=True)
        history["Stock"] = stock
        history = history.rename(columns={"Datetime": "Date"})
        history = history[["Stock", "Date", "Open", "High", "Low", "Close", "Volume"]]
        history = history[history["Date"] > start_datetime]
        all_data.append(history)
    return pd.concat(all_data, ignore_index=True)

# Load existing data from S3
def load_existing_data():
    try:
        response = s3_client.get_object(Bucket=S3_BUCKET, Key=S3_KEY)
        existing_df = pd.read_csv(response['Body'], parse_dates=["Date"])
        print(f"Loaded existing data: {len(existing_df)} rows")
        return existing_df
    except s3_client.exceptions.NoSuchKey:
        print("No existing data found in S3.")
        return None

# Upload DataFrame to S3
def upload_to_s3(df):
    csv_buffer = df.to_csv(index=False)
    s3_client.put_object(Bucket=S3_BUCKET, Key=S3_KEY, Body=csv_buffer)
    print(f"Uploaded updated dataset to S3: {S3_KEY}")

# Main logic
existing_df = load_existing_data()

if existing_df is not None:
    last_time = existing_df["Date"].max()
    new_data = fetch_stock_data(start_datetime=last_time)
    combined_df = pd.concat([existing_df, new_data], ignore_index=True)
    combined_df.drop_duplicates(subset=["Stock", "Date"], inplace=True)
else:
    print("First-time fetch for last 60 days (5m)...")
    start_time = datetime.now() - timedelta(days=60)
    combined_df = fetch_stock_data(start_datetime=start_time)

upload_to_s3(combined_df)


# Code to fetch the Historical News data for past 1 year:

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
import boto3
from io import StringIO

# AWS S3 Setup
S3_BUCKET = "data608-2025-stock-market-data"
S3_KEY = "news-data/historical_news_1yr.csv"
s3_client = boto3.client("s3")

# API Setup
API_KEY = 'hDdU807PlusyraTBnNgkkm2gFuPtkZ9F'
tickers = ["AAPL", "MSFT", "AMZN", "GOOGL", "META", "NVDA", "TSLA"]
tickers_str = ','.join(tickers)

def load_existing_news():
    try:
        response = s3_client.get_object(Bucket=S3_BUCKET, Key=S3_KEY)
        existing_df = pd.read_csv(response['Body'], parse_dates=["publishedDate"])
        print(f"Loaded existing news data: {len(existing_df)} articles")
        return existing_df
    except s3_client.exceptions.NoSuchKey:
        print("No existing news file found in S3.")
        return None

def upload_to_s3(df):
    buffer = StringIO()
    df.to_csv(buffer, index=False)
    s3_client.put_object(Bucket=S3_BUCKET, Key=S3_KEY, Body=buffer.getvalue())
    print(f"Updated news data uploaded to S3: {S3_KEY}")

def fetch_news(from_date, to_date):
    page = 0
    limit = 100
    all_news = []

    print(f"Fetching news from {from_date} to {to_date}")
    while True:
        page += 1
        print(f"Fetching page {page}")
        url = "https://financialmodelingprep.com/api/v3/stock_news"
        params = {
            "tickers": tickers_str,
            "from": from_date,
            "to": to_date,
            "limit": limit,
            "page": page,
            "apikey": API_KEY
        }

        response = requests.get(url, params=params)
        if response.status_code == 200:
            news_batch = response.json()
            if not news_batch:
                break
            all_news.extend(news_batch)
            time.sleep(1)
        else:
            print(f"Failed with status {response.status_code}")
            break

    if all_news:
        df = pd.DataFrame(all_news)
        df["publishedDate"] = pd.to_datetime(df["publishedDate"])
        df = df.sort_values(by=["symbol", "publishedDate"])
        return df
    else:
        print("No news returned.")
        return pd.DataFrame()

# Main logic
existing_news = load_existing_news()

if existing_news is not None:
    last_date = existing_news["publishedDate"].max()
    start_date = last_date.strftime('%Y-%m-%d')
    print(f"Updating from last saved date: {start_date}")
else:
    start_date = (datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d')
    print(f"First-time fetch from: {start_date}")
    
end_date = datetime.now().strftime('%Y-%m-%d')

# Fetch new data
new_news_df = fetch_news(start_date, end_date)

# Merge & upload
if not new_news_df.empty:
    if existing_news is not None:
        combined_df = pd.concat([existing_news, new_news_df], ignore_index=True)
        combined_df.drop_duplicates(subset=["symbol", "publishedDate", "title"], inplace=True)
    else:
        combined_df = new_news_df

    upload_to_s3(combined_df)
else:
    print("News already up to date. No new records.")


# The code to extract the real time news at the interval of 1 hour will be the same as shown above, which will give the latest news.

# Additionally, we will extract the real time stock data every 5 minutes, attach relative news with it, perform preprocessing and sentiment analysis, and finally store it in the database to train the maodel on the next day.

# Code for Sentiment analysis, pre processing and merging the stock news data with the news data:

Currently saving the merged data in s3 bucket, but for the real project, we will be storing the merged dataset in the Database

In [ ]:
import pandas as pd
import boto3
from io import StringIO
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.nn.functional import softmax
from tqdm import tqdm

# Load FinBERT
model_name = "yiyanghkust/finbert-tone"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# AWS S3 Config
bucket = "data608-2025-stock-market-data"
news_key = "historical-data/1_year_stock_news.csv"
stock_key = "historical-data/1_year_stock_data.csv"
output_key = "processed-data/merged_1yr_preprocessed.csv"

s3 = boto3.client("s3")

# Load news data from S3
news_obj = s3.get_object(Bucket=bucket, Key=news_key)
df = pd.read_csv(news_obj['Body'])
df = df.drop(columns=["image", "site", "url"], errors="ignore")
text_column = "text"

# Define FinBERT scoring function
def get_finbert_sentiment(text_column):
    try:
        inputs = tokenizer(text_column, return_tensors="pt", truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
            probs = softmax(outputs.logits, dim=1)
            sentiment_scores = probs[0].tolist()
            labels = ['negative', 'neutral', 'positive']
            sentiment_label = labels[torch.argmax(probs)]
            return sentiment_label, sentiment_scores
    except:
        return "error", [None, None, None]

tqdm.pandas()
df[['sentiment_label', 'sentiment_score']] = df[text_column].progress_apply(
    lambda x: pd.Series(get_finbert_sentiment(str(x)))
)

df[['neg_score', 'neu_score', 'pos_score']] = pd.DataFrame(df['sentiment_score'].tolist(), index=df.index)
df['sentiment_weighted'] = (-1 * df['neg_score']) + (0 * df['neu_score']) + (1 * df['pos_score'])
df.drop(columns=['sentiment_score', 'neg_score', 'neu_score', 'pos_score'], inplace=True)

# Aggregate sentiment by day and symbol
df['date'] = pd.to_datetime(df['publishedDate'], errors='coerce').dt.date
agg_df = df.groupby(['symbol', 'date']).agg({
    'sentiment_weighted': ['mean', 'min', 'max'],
    'sentiment_label': 'count'
}).reset_index()
agg_df.columns = ['symbol', 'date', 'sentiment_mean', 'sentiment_min', 'sentiment_max', 'sentiment_count']
agg_df.rename(columns={'symbol': 'Stock'}, inplace=True)

# Load stock data from S3
stock_obj = s3.get_object(Bucket=bucket, Key=stock_key)
stock_df = pd.read_csv(stock_obj['Body'])
stock_df['Date'] = pd.to_datetime(stock_df['Date'], utc=True, errors='coerce')
stock_df['Date'] = stock_df['Date'].dt.tz_convert(None)
stock_df['date'] = stock_df['Date'].dt.date

# Merge and save
merged_df = pd.merge(stock_df, agg_df, how="left", on=["Stock", "date"])

# Save back to S3
csv_buffer = StringIO()
merged_df.to_csv(csv_buffer, index=False)
s3.put_object(Bucket=bucket, Key=output_key, Body=csv_buffer.getvalue())

print("Merged data saved to S3:", output_key)


# Preprocess, sentiment, merge and store it to Database

In [ ]:
# ==============================
# Preprocess + Sentiment + Merge + Store to RDS
# ==============================

import pandas as pd
import boto3
from io import StringIO
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.nn.functional import softmax
from tqdm import tqdm
import psycopg2
from sqlalchemy import create_engine

# ========== 1. Load FinBERT ==========
model_name = "yiyanghkust/finbert-tone"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# ========== 2. AWS S3 Config ==========
bucket = "data608-2025-stock-market-data"
news_key = "historical-data/1_year_stock_news.csv"
stock_key = "historical-data/1_year_stock_data.csv"

s3 = boto3.client("s3")

# ========== 3. Load News Data ==========
news_obj = s3.get_object(Bucket=bucket, Key=news_key)
df_news = pd.read_csv(news_obj["Body"])
df_news = df_news.drop(columns=["image", "site", "url"], errors="ignore")

# ========== 4. Apply FinBERT ==========
def get_finbert_sentiment(text_column):
    try:
        inputs = tokenizer(text_column, return_tensors="pt", truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
            probs = softmax(outputs.logits, dim=1)
            sentiment_scores = probs[0].tolist()
            labels = ['negative', 'neutral', 'positive']
            sentiment_label = labels[torch.argmax(probs)]
            return sentiment_label, sentiment_scores
    except:
        return "error", [None, None, None]

tqdm.pandas()
df_news[["sentiment_label", "sentiment_score"]] = df_news["text"].progress_apply(
    lambda x: pd.Series(get_finbert_sentiment(str(x)))
)

df_news[["neg_score", "neu_score", "pos_score"]] = pd.DataFrame(df_news["sentiment_score"].tolist(), index=df_news.index)
df_news["sentiment_weighted"] = (-1 * df_news["neg_score"]) + (0 * df_news["neu_score"]) + (1 * df_news["pos_score"])
df_news.drop(columns=["sentiment_score", "neg_score", "neu_score", "pos_score"], inplace=True)

# ========== 5. Aggregate News ==========
df_news["date"] = pd.to_datetime(df_news["publishedDate"], errors="coerce").dt.date
agg_df = df_news.groupby(["symbol", "date"]).agg({
    "sentiment_weighted": ["mean", "min", "max"],
    "sentiment_label": "count"
}).reset_index()

agg_df.columns = ["symbol", "date", "sentiment_mean", "sentiment_min", "sentiment_max", "sentiment_count"]
agg_df.rename(columns={"symbol": "Stock"}, inplace=True)

# ========== 6. Load Stock Data ==========
stock_obj = s3.get_object(Bucket=bucket, Key=stock_key)
df_stock = pd.read_csv(stock_obj["Body"])
df_stock["Date"] = pd.to_datetime(df_stock["Date"], utc=True, errors="coerce")
df_stock["Date"] = df_stock["Date"].dt.tz_convert(None)
df_stock["date"] = df_stock["Date"].dt.date

# ========== 7. Merge ==========
merged_df = pd.merge(df_stock, agg_df, how="left", on=["Stock", "date"])

# ========== 8. Upload to PostgreSQL RDS ==========
# Replace these with your credentials
rds_host = "your-rds-endpoint.rds.amazonaws.com"
rds_port = "5432"
rds_db = "postgres"
rds_user = "admin"
rds_password = "your_password"

engine = create_engine(f'postgresql+psycopg2://{rds_user}:{rds_password}@{rds_host}:{rds_port}/{rds_db}')
merged_df.to_sql("historical_stock_sentiment", engine, if_exists="replace", index=False)

print("✅ Final merged dataset uploaded to RDS table: historical_stock_sentiment")


# Real-Time Daily Stock + News Update Script

In [ ]:
import pandas as pd
import requests
import psycopg2
from datetime import datetime, timedelta
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.nn.functional import softmax

# 🔧 Config
API_KEY = 'YOUR_FINANCIAL_MODELING_PREP_KEY'
stocks = ["AAPL", "MSFT", "AMZN", "GOOGL", "META", "NVDA", "TSLA"]
model_name = "yiyanghkust/finbert-tone"

# PostgreSQL RDS Config
RDS_HOST = 'your-db-endpoint.rds.amazonaws.com'
RDS_DB = 'your_db'
RDS_USER = 'your_user'
RDS_PASS = 'your_password'

# ⏳ Date
yesterday = datetime.utcnow().date() - timedelta(days=1)
yesterday_str = yesterday.strftime('%Y-%m-%d')

# 🔽 Fetch Daily Stock Data
import yfinance as yf
stock_rows = []
for symbol in stocks:
    data = yf.Ticker(symbol).history(period="2d", interval="1d").reset_index()
    if len(data) > 1:
        row = data.iloc[-1]
        stock_rows.append({
            "Stock": symbol,
            "Date": row["Date"].date(),
            "Open": row["Open"],
            "High": row["High"],
            "Low": row["Low"],
            "Close": row["Close"],
            "Volume": row["Volume"]
        })
stock_df = pd.DataFrame(stock_rows)

# 📰 Fetch News from Financial Modeling Prep
all_news = []
page = 0
limit = 100
while True:
    page += 1
    url = "https://financialmodelingprep.com/api/v3/stock_news"
    params = {
        "tickers": ','.join(stocks),
        "from": yesterday_str,
        "to": yesterday_str,
        "limit": limit,
        "page": page,
        "apikey": API_KEY
    }
    response = requests.get(url, params=params)
    if response.status_code != 200 or not response.json():
        break
    all_news.extend(response.json())

# ⬇️ Preprocess + Sentiment
if all_news:
    df = pd.DataFrame(all_news)
    df = df[["symbol", "publishedDate", "title"]].copy()
    df["publishedDate"] = pd.to_datetime(df["publishedDate"])
    df["date"] = df["publishedDate"].dt.date
    df["text"] = df["title"]

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

    def get_sentiment(text):
        try:
            inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
            with torch.no_grad():
                logits = model(**inputs).logits
                probs = softmax(logits, dim=1)[0]
                label = ['negative', 'neutral', 'positive'][torch.argmax(probs)]
                score = (-1 * probs[0].item()) + (0 * probs[1].item()) + (1 * probs[2].item())
                return label, score
        except:
            return "error", 0.0

    df[['sentiment_label', 'sentiment_score']] = df['text'].apply(lambda x: pd.Series(get_sentiment(str(x))))
    
    # Group sentiment
    sentiment_df = df.groupby(['symbol', 'date']).agg(
        sentiment_mean=('sentiment_score', 'mean'),
        sentiment_min=('sentiment_score', 'min'),
        sentiment_max=('sentiment_score', 'max'),
        sentiment_count=('sentiment_label', 'count')
    ).reset_index()
    sentiment_df.rename(columns={"symbol": "Stock"}, inplace=True)
else:
    # If no news, create neutral sentiment
    sentiment_df = pd.DataFrame({
        "Stock": stocks,
        "date": [yesterday]*len(stocks),
        "sentiment_mean": [0.0]*len(stocks),
        "sentiment_min": [0.0]*len(stocks),
        "sentiment_max": [0.0]*len(stocks),
        "sentiment_count": [0]*len(stocks)
    })

# 🔗 Merge
merged_df = pd.merge(stock_df, sentiment_df, left_on=["Stock", "Date"], right_on=["Stock", "date"], how="left")
merged_df.drop(columns=["date"], inplace=True)

# 💾 Store in PostgreSQL RDS
conn = psycopg2.connect(
    host=RDS_HOST,
    database=RDS_DB,
    user=RDS_USER,
    password=RDS_PASS
)
cursor = conn.cursor()

for _, row in merged_df.iterrows():
    cursor.execute("""
        INSERT INTO stock_sentiment_daily (
            stock, date, open, high, low, close, volume,
            sentiment_mean, sentiment_min, sentiment_max, sentiment_count
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (stock, date) DO NOTHING;
    """, (
        row['Stock'], row['Date'], row['Open'], row['High'], row['Low'],
        row['Close'], row['Volume'],
        row['sentiment_mean'], row['sentiment_min'],
        row['sentiment_max'], row['sentiment_count']
    ))

conn.commit()
cursor.close()
conn.close()

print("✅ Real-time stock + sentiment row inserted for", yesterday_str)


Code for the same, but will avoid duplication in the table:

In [ ]:
import pandas as pd
import requests
import psycopg2
from datetime import datetime, timedelta
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.nn.functional import softmax
import yfinance as yf

# 🔧 Config
API_KEY = 'YOUR_FINANCIAL_MODELING_PREP_KEY'
stocks = ["AAPL", "MSFT", "AMZN", "GOOGL", "META", "NVDA", "TSLA"]
model_name = "yiyanghkust/finbert-tone"

# PostgreSQL RDS Config
RDS_HOST = 'your-db-endpoint.rds.amazonaws.com'
RDS_DB = 'your_db'
RDS_USER = 'your_user'
RDS_PASS = 'your_password'

# ⏳ Date
yesterday = datetime.utcnow().date() - timedelta(days=1)
yesterday_str = yesterday.strftime('%Y-%m-%d')

# ────────────────────────────────
# 📈 Step 1: Fetch Stock Data (Daily)
stock_rows = []
for symbol in stocks:
    data = yf.Ticker(symbol).history(period="2d", interval="1d").reset_index()
    if len(data) > 1:
        row = data.iloc[-1]
        stock_rows.append({
            "Stock": symbol,
            "Date": row["Date"].date(),
            "Open": row["Open"],
            "High": row["High"],
            "Low": row["Low"],
            "Close": row["Close"],
            "Volume": row["Volume"]
        })

stock_df = pd.DataFrame(stock_rows)

# ────────────────────────────────
# 📰 Step 2: Fetch News (for yesterday only)
all_news = []
page = 0
limit = 100
while True:
    page += 1
    url = "https://financialmodelingprep.com/api/v3/stock_news"
    params = {
        "tickers": ','.join(stocks),
        "from": yesterday_str,
        "to": yesterday_str,
        "limit": limit,
        "page": page,
        "apikey": API_KEY
    }
    response = requests.get(url, params=params)
    if response.status_code != 200 or not response.json():
        break
    all_news.extend(response.json())

# ────────────────────────────────
# 🤖 Step 3: Sentiment Analysis
if all_news:
    df = pd.DataFrame(all_news)
    df = df[["symbol", "publishedDate", "title"]].copy()
    df["publishedDate"] = pd.to_datetime(df["publishedDate"])
    df["date"] = df["publishedDate"].dt.date
    df["text"] = df["title"]

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

    def get_sentiment(text):
        try:
            inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
            with torch.no_grad():
                logits = model(**inputs).logits
                probs = softmax(logits, dim=1)[0]
                score = (-1 * probs[0].item()) + (0 * probs[1].item()) + (1 * probs[2].item())
                return score
        except:
            return 0.0

    df["sentiment_score"] = df["text"].apply(lambda x: get_sentiment(str(x)))

    # Aggregate sentiment
    sentiment_df = df.groupby(['symbol', 'date']).agg(
        sentiment_mean=('sentiment_score', 'mean'),
        sentiment_min=('sentiment_score', 'min'),
        sentiment_max=('sentiment_score', 'max'),
        sentiment_count=('sentiment_score', 'count')
    ).reset_index()

    sentiment_df.rename(columns={"symbol": "Stock"}, inplace=True)
else:
    # Default to neutral sentiment if no news
    sentiment_df = pd.DataFrame({
        "Stock": stocks,
        "date": [yesterday] * len(stocks),
        "sentiment_mean": [0.0] * len(stocks),
        "sentiment_min": [0.0] * len(stocks),
        "sentiment_max": [0.0] * len(stocks),
        "sentiment_count": [0] * len(stocks)
    })

# ────────────────────────────────
# 🔗 Step 4: Merge
merged_df = pd.merge(stock_df, sentiment_df, left_on=["Stock", "Date"], right_on=["Stock", "date"], how="left")
merged_df.drop(columns=["date"], inplace=True)

# ────────────────────────────────
# 💾 Step 5: Store in PostgreSQL (UPSERT)
conn = psycopg2.connect(
    host=RDS_HOST,
    database=RDS_DB,
    user=RDS_USER,
    password=RDS_PASS
)
cursor = conn.cursor()

# Make sure UNIQUE constraint on (stock, date) is present in your table
for _, row in merged_df.iterrows():
    cursor.execute("""
        INSERT INTO stock_sentiment_daily (
            stock, date, open, high, low, close, volume,
            sentiment_mean, sentiment_min, sentiment_max, sentiment_count
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (stock, date) DO UPDATE SET
            open = EXCLUDED.open,
            high = EXCLUDED.high,
            low = EXCLUDED.low,
            close = EXCLUDED.close,
            volume = EXCLUDED.volume,
            sentiment_mean = EXCLUDED.sentiment_mean,
            sentiment_min = EXCLUDED.sentiment_min,
            sentiment_max = EXCLUDED.sentiment_max,
            sentiment_count = EXCLUDED.sentiment_count;
    """, (
        row['Stock'], row['Date'], row['Open'], row['High'], row['Low'],
        row['Close'], row['Volume'],
        row['sentiment_mean'], row['sentiment_min'],
        row['sentiment_max'], row['sentiment_count']
    ))

conn.commit()
cursor.close()
conn.close()

print(f"✅ Real-time stock + sentiment row stored for {yesterday_str}")


# preprocess, Sentiment analysis of 60 days 5 min data (MY Turn NOw)

In [ ]:
import pandas as pd
import boto3
from io import StringIO
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.nn.functional import softmax
from tqdm import tqdm
import psycopg2
from sqlalchemy import create_engine

# --- AWS & DB CONFIGURATION ---
S3_BUCKET = "data608-2025-stock-market-data"
STOCK_KEY = "real-time-stock-data/stock_60d_5m.csv"
NEWS_KEY = "real-time-news-data/historical_news_1yr.csv"
DB_NAME = "your_db_name"
DB_USER = "your_username"
DB_PASSWORD = "your_password"
DB_HOST = "your-db-host-name.ca-central-1.rds.amazonaws.com"
DB_PORT = "5432"
TABLE_NAME = "realtime_60day_merged"

# --- Load from S3 ---
s3 = boto3.client("s3")
stock_obj = s3.get_object(Bucket=S3_BUCKET, Key=STOCK_KEY)
news_obj = s3.get_object(Bucket=S3_BUCKET, Key=NEWS_KEY)

stock_df = pd.read_csv(stock_obj["Body"])
news_df = pd.read_csv(news_obj["Body"])

# --- Preprocessing Step 1: Datetime Alignment ---
stock_df["Date"] = pd.to_datetime(stock_df["Date"], utc=True)
stock_df["Date"] = stock_df["Date"].dt.tz_convert("America/New_York").dt.tz_localize(None)
news_df["publishedDate"] = pd.to_datetime(news_df["publishedDate"], errors="coerce")
stock_df["Date"] = stock_df["Date"].dt.floor("min")
news_df["publishedDate"] = news_df["publishedDate"].dt.floor("min")

# --- Filter news for same date range as stock ---
min_date = stock_df["Date"].min().date()
max_date = stock_df["Date"].max().date()
filtered_news = news_df[
    (news_df["publishedDate"].dt.date >= min_date) &
    (news_df["publishedDate"].dt.date <= max_date)
].copy()
filtered_news.rename(columns={"symbol": "Stock"}, inplace=True)

# --- Assign Latest News to Each Stock Row ---
stock_df.sort_values(["Stock", "Date"], inplace=True)
filtered_news.sort_values(["Stock", "publishedDate"], inplace=True)

assigned_news = []
for symbol, group in stock_df.groupby("Stock"):
    stock_times = group["Date"].values
    news_times = filtered_news[filtered_news["Stock"] == symbol][["publishedDate", "text"]].values
    latest_news = None
    news_index = 0
    for stock_time in stock_times:
        while news_index < len(news_times) and news_times[news_index][0] <= stock_time:
            latest_news = news_times[news_index][1]
            news_index += 1
        assigned_news.append(latest_news)

stock_df["latest_news"] = assigned_news

# --- Sentiment Analysis using FinBERT ---
model_name = "yiyanghkust/finbert-tone"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

def get_finbert_score(text):
    try:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
            probs = softmax(outputs.logits, dim=1)
            scores = probs[0].tolist()
            return -1 * scores[0] + 0 * scores[1] + 1 * scores[2]
    except:
        return 0.0

unique_news = stock_df["latest_news"].dropna().unique()
news_sentiment_map = {news: get_finbert_score(news) for news in tqdm(unique_news)}

# --- Sentiment Fading ---
faded_scores = []
fade_factor = 0.9
prev_news = None
prev_score = 0.0
repeat_count = 0

for news in stock_df["latest_news"]:
    if news != prev_news:
        sentiment = news_sentiment_map.get(news, 0.0)
        repeat_count = 0
    else:
        repeat_count += 1
        sentiment = prev_score * (fade_factor ** repeat_count)
    faded_scores.append(round(sentiment, 4))
    prev_news = news
    prev_score = news_sentiment_map.get(news, 0.0)

stock_df["faded_sentiment_score"] = faded_scores

# --- Store Final DataFrame to RDS PostgreSQL ---
print("Uploading final dataset to RDS PostgreSQL...")
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

# You can change if_exists to "replace" or "append" based on your use case
stock_df.to_sql(TABLE_NAME, engine, index=False, if_exists="replace")
print(f"✅ Table '{TABLE_NAME}' uploaded to RDS.")


# Real time stock data and news collection:

In [ ]:
import yfinance as yf
import requests
import pandas as pd
from datetime import datetime, timedelta
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.nn.functional import softmax
import psycopg2
from sqlalchemy import create_engine

# ───────────────────────────────────────────────
# 🔧 Config
API_KEY = "hDdU807PlusyraTBnNgkkm2gFuPtkZ9F"
tickers = ["AAPL", "MSFT", "AMZN", "GOOGL", "META", "NVDA", "TSLA"]
now = datetime.now()
lookback_days = 4
start_date = now - timedelta(days=lookback_days)

# RDS Connection (Update these)
DB_USER = "your_username"
DB_PASSWORD = "your_password"
DB_HOST = "your-db-hostname.amazonaws.com"
DB_PORT = "5432"
DB_NAME = "your_database"
TABLE_NAME = "latest_stock_sentiment"
conn_str = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(conn_str)

# ───────────────────────────────────────────────
# 📈 Step 1: Fetch Latest Stock Prices
stock_data = []
for ticker in tickers:
    df = yf.Ticker(ticker).history(period="1d", interval="5m")
    if not df.empty:
        latest_row = df.iloc[-1].copy()
        latest_row["Stock"] = ticker
        latest_row["Datetime"] = latest_row.name
        stock_data.append(latest_row)

stock_df = pd.DataFrame(stock_data)
stock_df.reset_index(drop=True, inplace=True)

# Convert to ET and make naive
stock_df["Datetime"] = pd.to_datetime(stock_df["Datetime"], utc=True)
stock_df["Datetime"] = stock_df["Datetime"].dt.tz_convert("America/New_York").dt.tz_localize(None)

# ───────────────────────────────────────────────
# 📰 Step 2: Fetch Latest News
def fetch_latest_news(ticker):
    url = "https://financialmodelingprep.com/api/v3/stock_news"
    params = {
        "tickers": ticker,
        "from": start_date.strftime("%Y-%m-%d"),
        "to": now.strftime("%Y-%m-%d"),
        "limit": 50,
        "apikey": API_KEY
    }
    try:
        response = requests.get(url, params=params)
        data = response.json()
        if isinstance(data, list) and data:
            return data[0]["text"]
    except Exception as e:
        print(f"Error fetching news for {ticker}: {e}")
    return None

stock_df["latest_news"] = stock_df["Stock"].apply(fetch_latest_news)

# ───────────────────────────────────────────────
# 🤖 Step 3: Sentiment Analysis
model_name = "yiyanghkust/finbert-tone"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

def get_weighted_sentiment(text):
    try:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
            probs = softmax(outputs.logits, dim=1)
            scores = probs[0].tolist()
            return round(-1 * scores[0] + 0 * scores[1] + 1 * scores[2], 4)
    except:
        return 0.0

stock_df["sentiment_score"] = stock_df["latest_news"].apply(
    lambda x: get_weighted_sentiment(str(x)) if pd.notna(x) else 0.0
)

# ───────────────────────────────────────────────
# 💾 Step 4: Save to PostgreSQL Table
try:
    stock_df.to_sql(TABLE_NAME, engine, if_exists="append", index=False)
    print(f"✅ Data saved to RDS table: {TABLE_NAME}")
except Exception as e:
    print(f"❌ Error saving to database: {e}")


# Moving the values from the table that contains only 1 day data to the database that contain the 60 days data:

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
from sqlalchemy import create_engine

# ────────────────────────
# Database connection details
DB_USER = "your_username"
DB_PASSWORD = "your_password"
DB_HOST = "your-db-host.amazonaws.com"
DB_PORT = "5432"
DB_NAME = "your_database"
SOURCE_TABLE = "latest_stock_sentiment"
TARGET_TABLE = "stock_sentiment_60d"

# ────────────────────────
# Setup DB connection
conn_str = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(conn_str)

# ────────────────────────
# Get yesterday's date
yesterday = (datetime.now() - timedelta(days=1)).date()

# Step 1: Read yesterday's real-time data
query = f"""
    SELECT * FROM {SOURCE_TABLE}
    WHERE DATE("Datetime") = '{yesterday}'
"""
df = pd.read_sql(query, engine)
print(f"✅ Fetched {len(df)} rows from {SOURCE_TABLE} for {yesterday}")

# Step 2: Append to 60-day table
if not df.empty:
    df.to_sql(TARGET_TABLE, engine, if_exists="append", index=False)
    print(f"✅ Appended data to {TARGET_TABLE}")

# Step 3: Optional - Remove rows older than 60 days
delete_query = f"""
    DELETE FROM {TARGET_TABLE}
    WHERE "Datetime" < NOW() - INTERVAL '60 days'
"""
with engine.connect() as conn:
    conn.execute(delete_query)
    print("🧹 Removed rows older than 60 days from target table")


# Code to load the 1 year stock data from the s3 bucket, clean the date column and then store it to a database:

In [ ]:
# ==============================
# Load 1-Year Stock Data → Clean → Upload to PostgreSQL
# ==============================

import pandas as pd
import boto3
from sqlalchemy import create_engine

# --- AWS S3 Configuration ---
bucket = "data608-2025-stock-market-data"
stock_key = "historical-stock-data/stock_1y_1d.csv"

s3 = boto3.client("s3")

# --- Load stock data from S3 ---
response = s3.get_object(Bucket=bucket, Key=stock_key)
df_stock = pd.read_csv(response["Body"])

# --- Clean the Date column ---
df_stock["Date"] = pd.to_datetime(df_stock["Date"], utc=True, errors="coerce")
df_stock["Date"] = df_stock["Date"].dt.tz_convert(None)  # Make timezone-naive
df_stock = df_stock.dropna(subset=["Date"])  # Drop any invalid rows

# Optional: Sort by stock and date
df_stock = df_stock.sort_values(["Stock", "Date"])

# --- PostgreSQL RDS Configuration ---
rds_host = "your-rds-endpoint.rds.amazonaws.com"
rds_port = "5432"
rds_db = "postgres"
rds_user = "admin"
rds_password = "your_password"
table_name = "historical_stock_1y_daily"

# --- Upload to PostgreSQL ---
engine = create_engine(f'postgresql+psycopg2://{rds_user}:{rds_password}@{rds_host}:{rds_port}/{rds_db}')
df_stock.to_sql(table_name, engine, if_exists="replace", index=False)

print(f"✅ Uploaded 1-year stock data to RDS table: {table_name}")


# Code to get the real time daily stock data (We will run this every morning)

In [ ]:
# ===========================
# Add New Daily Stock Data to 1-Year Table (No Sentiment)
# ===========================

import yfinance as yf
import pandas as pd
import psycopg2
from datetime import datetime, timedelta

# Config
stocks = ["AAPL", "MSFT", "AMZN", "GOOGL", "META", "NVDA", "TSLA"]
RDS_HOST = 'your-db-endpoint.rds.amazonaws.com'
RDS_DB = 'your_db'
RDS_USER = 'your_user'
RDS_PASS = 'your_password'
TABLE_NAME = "historical_stock_1y_daily"

# Get yesterday's date
yesterday = datetime.utcnow().date() - timedelta(days=1)

# Fetch yesterday's stock data
stock_rows = []
for symbol in stocks:
    data = yf.Ticker(symbol).history(period="2d", interval="1d").reset_index()
    if len(data) > 1:
        row = data.iloc[-1]
        stock_rows.append({
            "Stock": symbol,
            "Date": row["Date"].date(),
            "Open": row["Open"],
            "High": row["High"],
            "Low": row["Low"],
            "Close": row["Close"],
            "Volume": row["Volume"]
        })

# Insert into RDS
df = pd.DataFrame(stock_rows)
conn = psycopg2.connect(
    host=RDS_HOST,
    database=RDS_DB,
    user=RDS_USER,
    password=RDS_PASS
)
cursor = conn.cursor()

for _, row in df.iterrows():
    cursor.execute(f"""
        INSERT INTO {TABLE_NAME} (
            stock, date, open, high, low, close, volume
        ) VALUES (%s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (stock, date) DO NOTHING;
    """, (
        row['Stock'], row['Date'], row['Open'], row['High'],
        row['Low'], row['Close'], row['Volume']
    ))

conn.commit()
cursor.close()
conn.close()

print(f"✅ Daily stock prices for {yesterday} added to {TABLE_NAME}")


# Code to train the model every morning:

In [ ]:
# ===============================
# Train SARIMAX Models for Each Stock (60-Day, 5-Min Data)
# ===============================

import pandas as pd
import numpy as np
import joblib
from sqlalchemy import create_engine
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore")

# --- DB CONFIGURATION ---
DB_USER = "your_username"
DB_PASSWORD = "your_password"
DB_HOST = "your-db-host-name.ca-central-1.rds.amazonaws.com"
DB_PORT = "5432"
DB_NAME = "your_db_name"
TABLE_NAME = "realtime_60day_merged"

# --- Local Folder to Save Models ---
MODEL_SAVE_PATH = "./trained_models/"  # Or 's3://your-bucket/models/' for S3 version

# --- Best Model Orders (from evaluation) ---
best_model_orders = {
    "AAPL": (3, 1, 4),
    "META": (2, 1, 2),
    "NVDA": (0, 1, 0),
    "GOOGL": (1, 1, 0),
    "TSLA": (0, 1, 0),
    "AMZN": (1, 1, 0),
    "MSFT": (1, 1, 1)
}

# --- Connect to PostgreSQL and Load Data ---
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")
df = pd.read_sql(f"SELECT * FROM {TABLE_NAME}", engine)
df["Date"] = pd.to_datetime(df["Date"])
df.sort_values("Date", inplace=True)

# --- Feature Engineering ---
def add_features(df):
    df = df.copy()
    df['EMA_5'] = df['Close'].ewm(span=5, adjust=False).mean()
    df['EMA_12'] = df['Close'].ewm(span=12, adjust=False).mean()
    df['Volatility_30min'] = df['Close'].rolling(window=6).std()
    df['Price_Change_Pct'] = df['Close'].pct_change()
    df['Lag_Close'] = df['Close'].shift(1)
    df['Weekday'] = df['Date'].dt.weekday
    return df

# --- Train and Save Models ---
for stock in best_model_orders:
    print(f"\n📦 Training model for {stock}")
    stock_df = df[df["Stock"] == stock].copy()
    stock_df = add_features(stock_df)
    stock_df.dropna(inplace=True)
    stock_df.set_index("Date", inplace=True)

    features = ['Volume', 'faded_sentiment_score', 'EMA_5', 'EMA_12',
                'Volatility_30min', 'Price_Change_Pct', 'Lag_Close', 'Weekday']

    if len(stock_df) < 100:
        print(f"❌ Not enough data for {stock}, skipping...")
        continue

    y = stock_df['Close']
    X = stock_df[features]
    scaler = StandardScaler()
    X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns, index=X.index)

    order = best_model_orders[stock]

    try:
        model = SARIMAX(y, exog=X_scaled, order=order, enforce_stationarity=False)
        result = model.fit(disp=False)
        
        # Save model & scaler locally
        joblib.dump(result, f"{MODEL_SAVE_PATH}{stock}_sarimax.pkl")
        joblib.dump(scaler, f"{MODEL_SAVE_PATH}{stock}_scaler.pkl")
        print(f"✅ Saved model and scaler for {stock}")
        
    except Exception as e:
        print(f"❌ Error training model for {stock}: {e}")


Real-Time Data Fetch + Sentiment + Fading + Append to RDS Table + adding the feature engineering to the latest row + making predictions

In [ ]:
import yfinance as yf
import requests
import pandas as pd
from datetime import datetime, timedelta
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.nn.functional import softmax
from sqlalchemy import create_engine
from tqdm import tqdm
import joblib
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import StandardScaler
import os

# ──────────────── CONFIG ────────────────
API_KEY = "your_fmp_api_key"
tickers = ["AAPL", "MSFT", "AMZN", "GOOGL", "META", "NVDA", "TSLA"]
now = datetime.now()
fade_factor = 0.9

# DB Config
DB_USER = "your_user"
DB_PASSWORD = "your_password"
DB_HOST = "your-db-host.ca-central-1.rds.amazonaws.com"
DB_PORT = "5432"
DB_NAME = "your_db"
DATA_TABLE = "realtime_60day_merged"
PRED_TABLE = "realtime_60min_predictions"

# Model directory (each model is saved as {stock}.pkl)
MODEL_DIR = "models"

engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

# ──────────────── Step 1: Fetch Real-Time Stock ────────────────
stock_data = []
for ticker in tickers:
    df = yf.Ticker(ticker).history(period="1d", interval="5m")
    if not df.empty:
        latest_row = df.iloc[-1].copy()
        latest_row["Stock"] = ticker
        latest_row["Date"] = latest_row.name
        stock_data.append(latest_row)

stock_df = pd.DataFrame(stock_data)
stock_df.reset_index(drop=True, inplace=True)
stock_df["Date"] = pd.to_datetime(stock_df["Date"], utc=True)
stock_df["Date"] = stock_df["Date"].dt.tz_convert("America/New_York").dt.tz_localize(None)
stock_df["Date"] = stock_df["Date"].dt.floor("min")

# ──────────────── Step 2: Fetch Latest News ────────────────
def fetch_latest_news(ticker):
    url = "https://financialmodelingprep.com/api/v3/stock_news"
    params = {
        "tickers": ticker,
        "from": (now - timedelta(days=2)).strftime("%Y-%m-%d"),
        "to": now.strftime("%Y-%m-%d"),
        "limit": 50,
        "apikey": API_KEY
    }
    try:
        response = requests.get(url, params=params)
        data = response.json()
        if isinstance(data, list) and len(data) > 0:
            return data[0]["text"]
    except Exception as e:
        print(f"Error fetching news for {ticker}: {e}")
    return None

stock_df["latest_news"] = stock_df["Stock"].apply(fetch_latest_news)

# ──────────────── Step 3: Sentiment Analysis (FinBERT) ────────────────
model_name = "yiyanghkust/finbert-tone"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

def get_sentiment_score(text):
    try:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            logits = model(**inputs).logits
            probs = softmax(logits, dim=1)[0]
            score = -1 * probs[0].item() + 1 * probs[2].item()
            return round(score, 4)
    except:
        return 0.0

unique_news = stock_df["latest_news"].dropna().unique()
news_sentiment_map = {news: get_sentiment_score(news) for news in tqdm(unique_news)}

# ──────────────── Step 4: Sentiment Fading ────────────────
faded_scores = []
prev_news = None
prev_score = 0.0
repeat_count = 0

for news in stock_df["latest_news"]:
    if news != prev_news:
        sentiment = news_sentiment_map.get(news, 0.0)
        repeat_count = 0
    else:
        repeat_count += 1
        sentiment = prev_score * (fade_factor ** repeat_count)
    faded_scores.append(round(sentiment, 4))
    prev_news = news
    prev_score = news_sentiment_map.get(news, 0.0)

stock_df["faded_sentiment_score"] = faded_scores

# ──────────────── Step 5: Append Real-Time Row to Table ────────────────
stock_df.to_sql(DATA_TABLE, engine, if_exists="append", index=False)
print(f"✅ Real-time row appended to {DATA_TABLE}")

# ──────────────── Step 6: Forecast Using Last 50 Rows + Model ────────────────
def add_intraday_features(df):
    df = df.copy()
    df["EMA_5"] = df["Close"].ewm(span=5, adjust=False).mean()
    df["EMA_12"] = df["Close"].ewm(span=12, adjust=False).mean()
    df["Volatility_30min"] = df["Close"].rolling(window=6).std()
    df["Price_Change_Pct"] = df["Close"].pct_change()
    df["Lag_Close"] = df["Close"].shift(1)
    df["Weekday"] = df["Date"].dt.weekday
    return df

forecast_steps = {
    "15min": 3,
    "30min": 6,
    "1hour": 12
}

all_preds = []

for stock in tickers:
    df = pd.read_sql(f"SELECT * FROM {DATA_TABLE} WHERE \"Stock\" = '{stock}' ORDER BY \"Date\" DESC LIMIT 60", engine)
    df["Date"] = pd.to_datetime(df["Date"])
    df = df.sort_values("Date")
    df = add_intraday_features(df)

    feature_cols = [
        "Volume", "faded_sentiment_score",
        "EMA_5", "EMA_12", "Volatility_30min",
        "Price_Change_Pct", "Lag_Close", "Weekday"
    ]

    df = df[["Date", "Close"] + feature_cols].dropna()
    if len(df) < 20:
        print(f"⚠️ Not enough data for {stock} to predict.")
        continue

    endog = df["Close"]
    exog = df[feature_cols]
    exog_scaled = pd.DataFrame(StandardScaler().fit_transform(exog), columns=exog.columns, index=exog.index)

    try:
        model_path = os.path.join(MODEL_DIR, f"{stock}.pkl")
        saved_model = joblib.load(model_path)

        for label, steps in forecast_steps.items():
            forecast = saved_model.forecast(steps=steps, exog=exog_scaled.iloc[-steps:])
            forecast_mean = forecast.mean()
            current_price = endog.iloc[-1]
            direction = "Up" if forecast_mean > current_price else "Down"

            all_preds.append({
                "Stock": stock,
                "Forecast_Horizon": label,
                "Timestamp": endog.index[-1],
                "Current_Price": round(current_price, 4),
                "Forecast_Avg": round(forecast_mean, 4),
                "Predicted_Trend": direction
            })

    except Exception as e:
        print(f"❌ Prediction failed for {stock}: {e}")

# ──────────────── Step 7: Save Predictions ────────────────
if all_preds:
    pred_df = pd.DataFrame(all_preds)
    pred_df.to_sql(PRED_TABLE, engine, if_exists="append", index=False)
    print(f"✅ Predictions saved to {PRED_TABLE}")
else:
    print("❌ No predictions made.")
